In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("./data/wnjpn.db")

In [3]:
def get_word_id(word: str):
    cur = conn.execute("select wordid from word where lemma=(?)", (word, ))
    ret = cur.fetchone()
    return ret[0] if ret is not None else None

In [4]:
def get_synset_ids(word_id: int):
    cur = conn.execute("select synset from sense where wordid=(?)", (word_id, ))
    return [*map(lambda x: x[0], cur.fetchall())]

In [5]:
def get_synset_def(synset_id: str):
    cur = conn.execute("select def from synset_def where synset=(?) and lang='jpn'", (synset_id, ))
    return [*map(lambda x: x[0], cur.fetchall())]

In [6]:
def get_words_in_all_synsets(words: str or Iterable[str]):
    placeholder = "?" if type(words) is str else ",".join([
        "?" for i in range(0, len(words))
    ])
    params = (words,) if type (words) is str else words
    print(placeholder, params)
    cur = conn.execute(f"""\
select
    related_word.wordid,
    related_word.lemma
from word base
inner join sense attributed_sense
    on attributed_sense.wordid = base.wordid
inner join sense all_sense
    on all_sense.synset = attributed_sense.synset
inner join word related_word
    on related_word.wordid = all_sense.wordid
    and related_word.lang = "jpn"
where base.lemma in ({placeholder})""", words)
    return dict(map(lambda x:x[0:2], cur.fetchall()))


False

In [34]:
def get_words_in_synset(synset_id: str):
    cur = conn.execute("""\
select
    word.wordid,
    word.lemma 
from sense
inner join word
    on word.wordid = sense.wordid
    and word.lang = "jpn"
where synset=(?)"""
        , (synset_id, ))
    return dict(map(lambda x:x[0:2], cur.fetchall()))

#ids = get_synset_ids(get_word_id("優しい"))
#get_words_in_synset(ids[0])

In [9]:
def get_words_in_extended_synsets(word_id: int):
    sql=f"""\
select
    word.wordid,
    word.lemma
from sense base
left join synlink link
    on link.synset1 = base.synset
    and link.link = 'sim'
inner join sense extended
    on extended.synset = link.synset2
    or extended.synset = base.synset
inner join word
    on word.wordid = extended.wordid
    and word.lang = "jpn"
where base.wordid = (?)"""
    cur = conn.execute(sql, (word_id, ))
    return dict(map(lambda x:x[0:3], cur.fetchall()))

get_words_in_extended_synsets(get_word_id("優しい"))

{180094: '温か',
 195816: '細やか',
 199016: '暖かい',
 202105: '優しい',
 208523: '温い',
 224808: '暖か',
 230123: '温かい',
 170441: 'センチ',
 173304: '多情',
 181982: 'センティメンタル',
 182189: '多情多感',
 185046: '浪花節的',
 187641: 'ウエット',
 191515: '感情的',
 209229: '浪華節的',
 216783: 'ウェット',
 217954: '感傷的',
 245483: '心情的',
 245769: 'センチメンタル',
 223289: '温和',
 241360: '暖い',
 175797: '深切',
 176471: '世話好き',
 179535: '懇篤',
 180938: '懇ろ',
 181880: '親切',
 192504: '手厚い',
 193611: '親身',
 232581: '新切',
 236818: '篤い',
 240948: '厚い',
 159857: '柔か',
 190765: '静やか',
 194787: '柔かい',
 195414: '軟らかい',
 214659: '軟か',
 215954: '柔らか',
 223539: '柔い',
 230984: '軟かい',
 236122: 'ソフト',
 237902: '軟らか',
 175617: '小さい',
 212354: '鈍い',
 229136: '弱められた',
 163680: '物柔らか',
 171345: '円い',
 179621: '柔らかい',
 191588: '穏便',
 193395: '閑やか',
 193962: '穏和',
 198962: '穏やか',
 205072: 'もの柔か',
 206297: 'もの柔らか',
 219403: '物柔か',
 219515: '物静か',
 221200: 'もの静か',
 221855: 'たわやか',
 241943: '円か',
 248428: 'マイルド',
 164710: '静か',
 170642: '抑えた',
 164870: 'ちっこい',
 173

In [10]:
def get_words_in_extended_synsets(word_id: int):
    sql=f"""\
select
    word.wordid,
    word.lemma
from sense base
left join synlink link
    on link.synset1 = base.synset
    and link.link = 'sim'
inner join sense extended
    on extended.synset = link.synset2
    or extended.synset = base.synset
inner join word
    on word.wordid = extended.wordid
    and word.lang = "jpn"
where base.wordid = (?)"""
    cur = conn.execute(sql, (word_id, ))
    return dict(map(lambda x:x[0:3], cur.fetchall()))

get_words_in_extended_synsets(get_word_id("優しい"))

[('compassionate and kind; conciliatory; "he was soft on his children"',
  'also'),
 ('compassionate and kind', 'also'),
 ('あわれみ深く、親切な', 'also'),
 ('conciliatory', 'also'),
 ('懐柔的な', 'also'),
 ('showing care; "a protective mother"', 'sim'),
 ('showing care', 'sim'),
 ('心配を示すさま', 'sim'),
 ('given to or marked by sentiment or sentimentality', 'sim'),
 ('given to or marked by sentiment or sentimentality', 'sim'),
 ('感情や感傷を与えられる、または感情や感傷が特徴', 'sim'),
 ('feeling or showing love and affection; "loving parents"; "loving glances"',
  'sim'),
 ('feeling or showing love and affection', 'sim'),
 ('愛や愛情を感じるか、示すさま', 'sim'),
 ('having or showing a tender and considerate and helpful nature; used especially of persons and their behavior; "kind to sick patients"; "a kind master"; "kind words showing understanding and sympathy"; "thanked her for her kind letter"',
  'sim'),
 ('having or showing a tender and considerate and helpful nature', 'sim'),
 ('穏やかで思いやりがあり、有益である本質を持っているか、または示しているさま', 'sim'),
 ('us

In [ ]:
def get_hype_def(wordid: int):
    cur = conn.execute("""
select
    synset_def.def,
    link.link
from sense base
left join synlink link
   on link.synset1 = base.synset
left  join synset_def
   on synset_def.synset = link.synset2
where base.wordid = (?) and synset_def.lang='jpn'
"""
, (wordid, ))
    return [*map(lambda x: x[0:2], cur.fetchall())]

get_hype_def(get_word_id("優しい"))
